# Trade accounting

Performs the Borin and Mancini (2019) exports decomposition. It loads the MRIO tables stored in `data/mrio/`. Results are saved as `ta.parquet` in `data/`.

In [1]:
import numpy as np
import pandas as pd
import duckdb
from functions import subset, asvector, zeroout, diagvec, diagmat, diagrow

## Setup

In [2]:
# input = 'adb-mrio.parquet'
# years = np.arange(2017, 2022+1)
# output = 'ta'
# version = None

# input = 'adb-mrio62.parquet'
# years = np.append(2000, np.arange(2007, 2021+1))
# output = 'ta62'
# version = None

input = 'adb-mrio62-const.parquet'
years = np.arange(2007, 2022+1)
output = 'ta62-const'
version = None

# input = 'adb-mrio_jun2023.parquet'
# years = np.arange(2020, 2022+1)
# output = 'ta'
# version = 'jun2023'

sectors = pd.read_excel('../data/raw/sectors.xlsx')
sectors = sectors.drop_duplicates(subset='ind', ignore_index=True)

# G = 73      # Number of countries + ROW
G = 63      
N = 35      # Number of sectors
f = 5       # Number of final demand components

np.seterr(divide='ignore', invalid='ignore')

{'divide': 'warn', 'over': 'warn', 'under': 'ignore', 'invalid': 'warn'}

## Decompositions

Note that the country index `s` used henceforth corresponds to the MRIO country indices, which start at 1 and not 0.

In [3]:
ta = pd.DataFrame()
ta_es = pd.DataFrame()
ta_os = pd.DataFrame()

for year in years:
    
    mrio = duckdb.sql(f"SELECT * EXCLUDE(t, si) FROM read_parquet('../data/mrio/{input}') WHERE t={year}").df()
    mrio = mrio.values

    x = mrio[-1][:(G*N)]
    Z = mrio[:(G*N)][:, :(G*N)]
    va = np.sum(mrio[-7:-1][:, :(G*N)], axis=0)
    Y_big = mrio[:(G*N)][:, (G*N):-1]
    Y = Y_big @ np.kron(np.eye(G), np.ones((f, 1)))
    Yd = zeroout(Y, inverse=True)
    Yf = zeroout(Y)
    v = np.where(x != 0, va/x, 0)
    Dx = np.diag(np.where(x != 0, 1/x, 0))
    A = Z @ Dx
    Ad = zeroout(A, inverse=True)
    Af = zeroout(A)
    B = np.linalg.inv(np.eye(G*N) - A)
    Bd = np.linalg.inv(np.eye(G*N) - Ad)
    E = zeroout(Z @ np.kron(np.eye(G), np.ones((N, 1))) + Y)

    for s in range(1, G+1):

        # Country level

        Exports = np.sum(subset(E, s, -s), axis=0)
        Bnots = np.linalg.inv(np.eye(G*N) - zeroout(A, s, -s))
        VB_DC = subset(v, s) @ subset(Bnots, s, s)
        VB_FC = subset(v, -s) @ subset(Bnots, -s, s)
        DAVAX1 = VB_DC @ subset(Y, s, -s)
        DAVAX2 = VB_DC @ subset(A, s, -s) @ subset(Bd, -s, -s) @ subset(Yd, -s, -s)
        REX1 = VB_DC @ subset(A, s, -s) @ subset(Bd, -s, -s) @ diagvec(np.sum(subset(Yf, -s, -s), axis=1))
        REX2 = VB_DC @ subset(A, s, -s) @ subset(Bd, -s, -s) @ diagmat(subset(Af, -s, 0) @ B @ subset(Y, 0, -s), offd=True)
        REX3 = VB_DC @ subset(A, s, -s) @ subset(Bd, -s, -s) @ diagmat(subset(Af, -s, 0) @ B @ subset(Y, 0, -s))
        REF1 = VB_DC @ subset(A, s, -s) @ subset(Bd, -s, -s) @ diagvec(subset(Yf, -s, s))
        REF2 = VB_DC @ subset(A, s, -s) @ subset(Bd, -s, -s) @ diagvec(subset(Af, -s, 0) @ B @ subset(Y, 0, s))
        FVA = VB_FC @ subset(E, s, -s)
        PDC1 = VB_DC @ subset(Af, s, 0) @ subset(B, 0, s) @ subset(E, s, -s)
        PDC2 = VB_FC @ subset(Af, s, 0) @ subset(B, 0, s) @ subset(E, s, -s)

        ta_s = pd.DataFrame({
            't': year, 's': s, 'r': np.setdiff1d(np.arange(1, G+1), s),
            'breakdown': 'none', 'i': 0, 'i5': 0, 'i15': 0,
            'Exports': Exports,
            'DAVAX1': DAVAX1, 'DAVAX2': DAVAX2,
            'REX1': REX1, 'REX2': REX2, 'REX3': REX3,
            'REF1': REF1, 'REF2': REF2,
            'FVA': FVA, 'PDC1': PDC1, 'PDC2': PDC2
        })
        ta = pd.concat([ta, ta_s], ignore_index=True)

        # Breakdown by export sectors

        Exports = subset(E, s, -s)
        Bnots = np.linalg.inv(np.eye(G*N) - zeroout(A, s, -s))
        VB_DC = np.diag(subset(v, s) @ subset(Bnots, s, s))
        VB_FC = np.diag(subset(v, -s) @ subset(Bnots, -s, s))
        DAVAX1 = VB_DC @ subset(Y, s, -s)
        DAVAX2 = VB_DC @ subset(A, s, -s) @ subset(Bd, -s, -s) @ subset(Yd, -s, -s)
        REX1 = VB_DC @ subset(A, s, -s) @ subset(Bd, -s, -s) @ diagvec(np.sum(subset(Yf, -s, -s), axis=1))
        REX2 = VB_DC @ subset(A, s, -s) @ subset(Bd, -s, -s) @ diagmat(subset(Af, -s, 0) @ B @ subset(Y, 0, -s), offd=True)
        REX3 = VB_DC @ subset(A, s, -s) @ subset(Bd, -s, -s) @ diagmat(subset(Af, -s, 0) @ B @ subset(Y, 0, -s))
        REF1 = VB_DC @ subset(A, s, -s) @ subset(Bd, -s, -s) @ diagvec(subset(Yf, -s, s))
        REF2 = VB_DC @ subset(A, s, -s) @ subset(Bd, -s, -s) @ diagvec(subset(Af, -s, 0) @ B @ subset(Y, 0, s))
        FVA = VB_FC @ subset(E, s, -s)
        PDC1 = np.diag(subset(v, s) @ subset(Bnots, s, s) @ subset(Af, s, 0) @ subset(B, 0, s)) @ subset(E, s, -s)
        PDC2 = np.diag(subset(v, -s) @ subset(Bnots, -s, s) @ subset(Af, s, 0) @ subset(B, 0, s)) @ subset(E, s, -s)

        ta_es_s = pd.DataFrame({
            't': year, 's': s, 'r': np.setdiff1d(np.arange(1, G+1), s).repeat(N),
            'breakdown': 'es',
            'i': np.tile(sectors['ind'], G-1),
            'i5': np.tile(sectors['ind5'], G-1),
            'i15': np.tile(sectors['ind15'], G-1),
            'Exports': asvector(Exports),
            'DAVAX1': asvector(DAVAX1), 'DAVAX2': asvector(DAVAX2),
            'REX1': asvector(REX1), 'REX2': asvector(REX2), 'REX3': asvector(REX3),
            'REF1': asvector(REF1), 'REF2': asvector(REF2),
            'FVA': asvector(FVA), 'PDC1': asvector(PDC1), 'PDC2': asvector(PDC2)
        })
        ta_es = pd.concat([ta_es, ta_es_s], ignore_index=True)

        # Breakdown by origin sectors

        VB_DC = np.diag(subset(v, s)) @ subset(Bnots, s, s)
        VB_FC = diagrow(subset(v, -s)) @ subset(Bnots, -s, s)
        DAVAX1 = VB_DC @ subset(Y, s, -s)
        DAVAX2 = VB_DC @ subset(A, s, -s) @ subset(Bd, -s, -s) @ subset(Yd, -s, -s)
        REX1 = VB_DC @ subset(A, s, -s) @ subset(Bd, -s, -s) @ diagvec(np.sum(subset(Yf, -s, -s), axis=1))
        REX2 = VB_DC @ subset(A, s, -s) @ subset(Bd, -s, -s) @ diagmat(subset(Af, -s, 0) @ B @ subset(Y, 0, -s), offd=True)
        REX3 = VB_DC @ subset(A, s, -s) @ subset(Bd, -s, -s) @ diagmat(subset(Af, -s, 0) @ B @ subset(Y, 0, -s))
        REF1 = VB_DC @ subset(A, s, -s) @ subset(Bd, -s, -s) @ diagvec(subset(Yf, -s, s))
        REF2 = VB_DC @ subset(A, s, -s) @ subset(Bd, -s, -s) @ diagvec(subset(Af, -s, 0) @ B @ subset(Y, 0, s))
        FVA = VB_FC @ subset(E, s, -s)
        PDC1 = VB_DC @ subset(Af, s, 0) @ subset(B, 0, s) @ subset(E, s, -s)
        PDC2 = VB_FC @ subset(Af, s, 0) @ subset(B, 0, s) @ subset(E, s, -s)

        ta_os_s = pd.DataFrame({
            't': year, 's': s, 'r': np.setdiff1d(np.arange(1, G+1), s).repeat(N),
            'breakdown': 'os',
            'i': np.tile(sectors['ind'], G-1), 'i5': np.tile(sectors['ind5'], G-1), 'i15': np.tile(sectors['ind15'], G-1),
            'Exports': asvector(Exports),
            'DAVAX1': asvector(DAVAX1), 'DAVAX2': asvector(DAVAX2),
            'REX1': asvector(REX1), 'REX2': asvector(REX2), 'REX3': asvector(REX3),
            'REF1': asvector(REF1), 'REF2': asvector(REF2),
            'FVA': asvector(FVA), 'PDC1': asvector(PDC1), 'PDC2': asvector(PDC2)
        })
        ta_os = pd.concat([ta_os, ta_os_s], ignore_index=True)

    print(f'{year} done')

2007 done
2008 done
2009 done
2010 done
2011 done
2012 done
2013 done
2014 done
2015 done
2016 done
2017 done
2018 done
2019 done
2020 done
2021 done
2022 done


In [4]:
ta = pd.concat([ta, ta_es], ignore_index=True)
ta = pd.concat([ta, ta_os], ignore_index=True)

if version is None:
    outputname = f'{output}.parquet'
else:
    outputname = f'{output}_{version}.parquet'

ta.to_parquet(f'../data/{outputname}', index=False)

In [5]:
ta

,t,s,r,breakdown,i,i5,i15,Exports,DAVAX1,DAVAX2,REX1,REX2,REX3,REF1,REF2,FVA,PDC1,PDC2
0,2007,1,2,none,0,0,0,196.833563,42.700026,58.122286,25.922877,43.291390,0.539014,0.227090,0.359708,25.416129,0.217976,0.037066
1,2007,1,3,none,0,0,0,1506.921005,143.096488,277.729350,170.330921,627.832676,6.890348,0.899820,4.957448,272.430063,2.357243,0.396648
2,2007,1,4,none,0,0,0,51.265191,9.335753,22.877417,3.802484,6.711518,0.013894,0.013890,0.040709,8.383602,0.073410,0.012514
3,2007,1,5,none,0,0,0,2158.418354,250.236953,1290.633779,106.409248,257.557910,1.570588,0.505763,2.630989,246.406386,2.127610,0.339130
4,2007,1,6,none,0,0,0,3159.475444,1219.539027,768.766297,221.833764,434.566941,10.371791,1.528349,3.630892,494.377921,4.156783,0.703679
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4437211,2022,63,62,os,31,5,13,3.191044,4.927599,13.654984,3.507212,8.820481,0.026313,1.640732,2.919619,21.639514,0.570396,0.240281
4437212,2022,63,62,os,32,5,14,10.872655,2.346731,7.663171,2.081514,4.594646,0.013403,1.014360,1.550625,9.160454,0.228167,0.096151
4437213,2022,63,62,os,33,5,14,5.240416,3.346332,7.031566,1.720621,4.231752,0.012586,0.806879,1.399956,3.813214,0.203002,0.042789
4437214,2022,63,62,os,34,5,15,47.515597,12.586586,26.087043,7.010145,16.200011,0.047329,3.328490,5.342665,24.042701,0.766416,0.262457
